In [90]:
import pandas as pd

df = pd.read_csv('../Our Datasets/features_sample_match.csv')

start_types_wants = ['pass_interception', 'recovery']

df = df[df['start_type'].isin(start_types_wants)]

# dropping irrelevant columns
cols_to_drop = ['Unique ID', 'match_id', 'event_index', 'frame_anchor', 'rec_player_id', 'rec_team_short', 'start_type', 'end_type', 'event_row_index', 'source_file', 'error', 'third_end',
                'game_state', 'mean_team_dist', 'dist_to_near_goal', 'dist_to_far_goal', 'd_nearest_team']

df = df.drop(columns=cols_to_drop)

#df['team_out_of_possession_phase_type'] = df['team_out_of_possession_phase_type'].str.replace('high_block', 'block')
#df['team_out_of_possession_phase_type'] = df['team_out_of_possession_phase_type'].str.replace('medium_block', 'block')
#df['team_out_of_possession_phase_type'] = df['team_out_of_possession_phase_type'].str.replace('low_block', 'block')

In [91]:
df

,dist_to_attacking_goal,d_nearest_opp,n_opp_within5,n_forward_options,team_out_of_possession_phase_type,third_start,max_player_targeted_xthreat
0,99.575170,7.026308,0.0,10.0,high_block,defensive,0.0024
3,34.427403,0.000000,1.0,9.0,chaotic,attacking,0.0091
9,30.598440,4.389533,1.0,0.0,chaotic,attacking,0.0091
14,51.188596,2.489598,2.0,7.0,chaotic,middle,0.0359
19,57.972010,0.982700,1.0,8.0,chaotic,middle,0.0359
...,...,...,...,...,...,...,...
5850,82.279522,5.026659,0.0,6.0,chaotic,defensive,NaN
5851,59.568519,6.901311,0.0,9.0,chaotic,middle,NaN
5852,34.236218,5.721416,0.0,6.0,chaotic,attacking,0.0133
5853,20.692561,6.528714,0.0,8.0,disruption,attacking,NaN


In [92]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRegressor  # pip install xgboost if needed

In [93]:
#third_order = ["defensive", "middle", "attacking"]
#df["third_start"] = df["third_start"].map({v: i for i, v in enumerate(third_order)})

df = df[['dist_to_attacking_goal', 'max_player_targeted_xthreat', 'team_out_of_possession_phase_type', 'n_forward_options']]

game_state_map = {
    "losing": -1,
    "drawing": 0,
    "winning": 1
}
#df["game_state"] = df["game_state"].map(game_state_map)

df = pd.get_dummies(df, columns=["team_out_of_possession_phase_type"], prefix="oop_phase")

target_col = "max_player_targeted_xthreat"
df = df[df[target_col].notna()]

In [94]:
df

,dist_to_attacking_goal,max_player_targeted_xthreat,n_forward_options,oop_phase_chaotic,oop_phase_defending_direct,oop_phase_defending_quick_break,oop_phase_defending_set_play,oop_phase_defending_transition,oop_phase_disruption,oop_phase_high_block,oop_phase_low_block,oop_phase_medium_block
0,99.575170,0.0024,10.0,False,False,False,False,False,False,True,False,False
3,34.427403,0.0091,9.0,True,False,False,False,False,False,False,False,False
9,30.598440,0.0091,0.0,True,False,False,False,False,False,False,False,False
14,51.188596,0.0359,7.0,True,False,False,False,False,False,False,False,False
19,57.972010,0.0359,8.0,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
5844,84.239471,0.0006,5.0,False,True,False,False,False,False,False,False,False
5845,71.880025,0.0064,2.0,False,False,False,False,False,True,False,False,False
5847,60.018211,0.0014,4.0,False,False,False,False,False,True,False,False,False
5848,78.036013,0.0026,2.0,True,False,False,False,False,False,False,False,False


In [95]:
df_model = df.copy()

target_col = "max_player_targeted_xthreat"
df_model = df_model[df_model[target_col].notna()]

X = df_model.drop(columns=["max_player_targeted_xthreat"])
y = df_model["max_player_targeted_xthreat"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42
)

In [96]:
rf = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [97]:
xgb = XGBRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1,
    objective="reg:squarederror"
)
xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=-1, num_parallel_tree=None, ...)

In [98]:
def evaluate(model, X_train, y_train, X_test, y_test, name):
    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)

    print(f"\n📌 {name} Performance")
    print(f"Train RMSE: {np.sqrt(mean_squared_error(y_train, pred_train))} | "
          f"R2: {r2_score(y_train, pred_train):.3f}")
    print(f"Test  RMSE: {np.sqrt(mean_squared_error(y_test, pred_test))} | "
          f"R2: {r2_score(y_test, pred_test):.3f}")

evaluate(rf, X_train, y_train, X_test, y_test, "Random Forest")
evaluate(xgb, X_train, y_train, X_test, y_test, "XGBoost")


📌 Random Forest Performance
Train RMSE: 0.014165908455157196 | R2: 0.856
Test  RMSE: 0.05107892684665925 | R2: 0.063

📌 XGBoost Performance
Train RMSE: 0.00813212874340982 | R2: 0.952
Test  RMSE: 0.052392149609132375 | R2: 0.015


In [99]:
y_mean = np.full_like(y_test, y_train.mean(), dtype=float)

baseline_rmse = np.sqrt(mean_squared_error(y_test, y_mean))
baseline_r2   = r2_score(y_test, y_mean)

print("Baseline RMSE:", baseline_rmse)
print("Baseline R2:", baseline_r2)

Baseline RMSE: 0.052834685465648804
Baseline R2: -0.00221018682994667


In [100]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=0.1)
ridge.fit(X_train, y_train)
evaluate(ridge, X_train, y_train, X_test, y_test, "Ridge")


📌 Ridge Performance
Train RMSE: 0.03537263712458906 | R2: 0.100
Test  RMSE: 0.05139106609724742 | R2: 0.052


In [101]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
linreg.fit(X_train, y_train)

evaluate(linreg, X_train, y_train, X_test, y_test, "Linear Regression")


📌 Linear Regression Performance
Train RMSE: 0.03537262719298078 | R2: 0.100
Test  RMSE: 0.05139183857691296 | R2: 0.052
